In [1]:
from urllib.request import Request, urlopen
import urllib.request
from bs4 import BeautifulSoup
import os
import pandas as pd

import mysql.connector
import os

from datetime import date, datetime

In [2]:
timestamp = datetime.now()
print("Execution time: ", timestamp.strftime("%d/%m/%Y %H:%M:%S"))

Execution time:  12/05/2022 14:37:03


In [3]:
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

opener = AppURLopener()

/tmp/ipykernel_20198/2111673530.py:4: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


In [4]:
def call_web(_source):
    
    req = Request(_source, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'})
    return urlopen(req, timeout=10).read()

def parse_url(_raw):
    soupdata =BeautifulSoup(_raw, "html.parser")
    
    tables = soupdata.find_all("tr")
    
    #tables = soupdata.find_all('div')
    return tables

In [5]:
def get_data():
    
    raw_url = call_web('https://www.flightradar24.com/data/airlines')
    parsed =  parse_url(raw_url)
    
    raw_elements = []
    err_elements = []

    for line in parsed:
        row = line.find_all('td')
        
        raw_elements.append(row)
        
        

In [6]:
def get_img(_html):
    html = str(_html).split('" ')
    
    img = False
    result = 'placeholder'

    for tg in html:
    
        if tg[:12] == 'data-bn-lazy':
            result = tg[18:]
            img = True
        else:
            mf = False
    
    return result


In [7]:
raw_url = call_web('https://www.flightradar24.com/data/airlines')
parsed =  parse_url(raw_url)

In [8]:
raw_elm = []

for row in parsed:
    fields = row.find_all('td')
    if len(fields) == 6:
        raw_elm.append(fields)
    

In [9]:
std_elm = []
for airl in raw_elm:
    
    row = []
    
    row.append(get_img(airl[1]))
    
    for elm in airl[2:-1]:
        
        row.append(elm.get_text())
    
    std_elm.append(row)
        
    

In [10]:
curated_elm = []

for elm in std_elm:
    
    row = []
    iata = None
    oaci = None
    row.append(elm[1])
    
    codes = elm[2].split('/')
    for code in codes:
        #print(code, len(list(code)))
        if len(list(code)) == 3:
            iata = code.strip()
        else:
            oaci = code.strip()
    
    row.append(iata)
    row.append(oaci)
    row.append(int(str(elm[3].split(' ')[0]).strip()))
    row.append(elm[0])
    row.append(timestamp)
    
    curated_elm.append(row)
    

In [11]:
table = pd.DataFrame(curated_elm, columns=['name', 'iata', 'oaci', 'fleet_size', 'img_ext', 'timestamp'])
table.to_csv('airlines_flightradar24.csv')

In [18]:
table.iloc[10]['img_ext']

'https://s3.eu-central-1.amazonaws.com/images.flightradar24.com/assets/airlines/logotypes/A3_AEE.png'